<a href="https://www.kaggle.com/code/ayushkhaire/top-1000-cryptos-daily-updates?scriptVersionId=186282000" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import requests as rq
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
from kaggle_secrets import UserSecretsClient
from datetime import datetime
import json
import subprocess
import logging
import time

# Yeasterday data :)

In [2]:
df = pd.read_csv('/kaggle/input/top-1000-cryptos-historical/1000_cryptos.csv')
df

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,0,2017-11-09,0.025160,0.035060,0.025006,0.032053,0.032053,18716200.0,BTC-USD
1,1,2017-11-10,0.032219,0.033348,0.026451,0.027119,0.027119,6766780.0,BTC-USD
2,2,2017-11-11,0.026891,0.029659,0.025684,0.027437,0.027437,5532220.0,BTC-USD
3,3,2017-11-12,0.027480,0.027952,0.022591,0.023977,0.023977,7280250.0,BTC-USD
4,4,2017-11-13,0.024364,0.026300,0.023495,0.025808,0.025808,4419440.0,BTC-USD
...,...,...,...,...,...,...,...,...,...
1127504,1127504,2024-06-25,18.700827,19.214235,18.614672,18.913424,18.913424,3191399.0,JUV-USD
1127505,1127505,2024-06-26,18.913202,19.211378,17.585800,17.601185,17.601185,4326273.0,JUV-USD
1127506,1127506,2024-06-27,17.597841,17.929531,17.096172,17.273285,17.273285,4391237.0,JUV-USD
1127507,1127507,2024-06-28,17.274357,17.849468,16.767269,16.910219,16.910219,5492596.0,JUV-USD


# Define headers

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Extract symbols

In [4]:
 link = 'https://finance.yahoo.com/crypto/?offset=0&count=100'

# Get symbols and hit urls

In [5]:
resp_main_list = []
i = 0
for page in tqdm(range(10)):
    url = f'https://finance.yahoo.com/crypto/?offset={i*100}&count=100'
    r = rq.get(url,headers = headers)
    resp_main_list.append(r)
    i = i+1
#     print(url)

100%|██████████| 10/10 [00:15<00:00,  1.56s/it]


In [6]:
symbol_list = []

for r in resp_main_list:
    soup = BeautifulSoup(r.text,'html.parser')
    symbs= soup.find_all('a',{'class':'Fw(600) C($linkColor)'})
    for s in symbs:
        symbol_list.append(s.text)
        
print(len(symbol_list))

1000


In [7]:
unique_symbol_list = []

for sy in symbol_list:
    if sy not in unique_symbol_list:
        unique_symbol_list.append(sy)

unique_symbol_list = list(set(unique_symbol_list))
print(len(unique_symbol_list))

1000


# Now , get the link for today downloads from a tesla stock 

### This we willl use to use "code" by yahoo finance to figure out dates 

In [8]:
'''

link1 = https://query1.finance.yahoo.com/v7/finance/download/TSLA?period1=1684241704&period2=1715864104&interval=1d&events=history&includeAdjustedClose=true

link2 = https://query1.finance.yahoo.com/v7/finance/download/AMD?period1=1684241769&period2=1715864169&interval=1d&events=history&includeAdjustedClose=true


here the period values are exactly same.
'''

# After research , i got they are unix timestamps in period . lets try it out !

import time

# Get current Unix timestamp
current_timestamp = int(time.time())

date_str = "2015-01-01"
date_obj = datetime.strptime(date_str, "%Y-%m-%d")
period1 = int(time.mktime(date_obj.timetuple()))


period2 = current_timestamp  

period1, period2 = period2, period1

# Print the timestamps in the desired format
print(f"period1={period1}&period2={period2}")


period1=1719787488&period2=1420070400


# create stocks directory

In [9]:
os.mkdir('crypto_data')

# Download all files

In [10]:
for stock in tqdm(symbol_list[:1000]):
    url = f"https://query1.finance.yahoo.com/v7/finance/download/{stock}?period1={period2}&period2={period1}&interval=1d&events=history&includeAdjustedClose=true"
#     print(url)
    r = rq.get(url, headers=headers)
    if r.status_code == 200:
        with open(f"crypto_data/{stock}.csv", "wb") as file:
            file.write(r.content)
#         print(f"File downloaded successfully : {stock}.csv")
#     else:
#         print(f"Failed to download file : {stock}.csv")
#     time.sleep(3)


# https://query1.finance.yahoo.com/v7/finance/download/AMD?period1=1684244457&period2=1715866857&interval=1d&events=history&includeAdjustedClose=true

100%|██████████| 1000/1000 [01:42<00:00,  9.80it/s]


# Read a sample

In [11]:
first_file = os.listdir('/kaggle/working/crypto_data')[0]
print(first_file)
df = pd.read_csv(f'/kaggle/working/crypto_data/{first_file}')
df

ORAI-USD.csv


,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-10-28,9.946349,10.096945,3.507278,4.387400,4.387400,1444282
1,2020-10-29,4.387630,9.351766,2.831383,7.534497,7.534497,2058600
2,2020-10-30,7.534420,9.034474,5.401276,7.800243,7.800243,1280547
3,2020-10-31,7.800292,8.312695,5.751888,6.257072,6.257072,350813
4,2020-11-01,6.257779,7.355800,5.695908,7.217144,7.217144,177058
...,...,...,...,...,...,...,...
1337,2024-06-26,9.156507,9.186596,8.948361,8.995871,8.995871,3961494
1338,2024-06-27,8.995836,9.233989,8.880602,9.118688,9.118688,4280548
1339,2024-06-28,9.119044,9.208364,8.389455,8.399383,8.399383,5013629
1340,2024-06-29,8.398866,8.739178,8.398866,8.566529,8.566529,4388104


# Work on dataset

In [12]:
csvlist = os.listdir('/kaggle/working/crypto_data')

In [13]:
megadf = pd.DataFrame()

In [14]:
for sym, file in zip(symbol_list, csvlist):
    df = pd.read_csv(f'/kaggle/working/crypto_data/{file}')
    df['Symbol'] = sym
    megadf = pd.concat([megadf, df], ignore_index=True)

In [15]:
megadf

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,2020-10-28,9.946349,10.096945,3.507278,4.387400,4.387400,1444282.0,BTC-USD
1,2020-10-29,4.387630,9.351766,2.831383,7.534497,7.534497,2058600.0,BTC-USD
2,2020-10-30,7.534420,9.034474,5.401276,7.800243,7.800243,1280547.0,BTC-USD
3,2020-10-31,7.800292,8.312695,5.751888,6.257072,6.257072,350813.0,BTC-USD
4,2020-11-01,6.257779,7.355800,5.695908,7.217144,7.217144,177058.0,BTC-USD
...,...,...,...,...,...,...,...,...
1130860,2024-06-26,0.000000,0.000000,0.000000,0.000000,0.000000,426785.0,QTCON-USD
1130861,2024-06-27,0.000000,0.000000,0.000000,0.000000,0.000000,320872.0,QTCON-USD
1130862,2024-06-28,0.000000,0.000000,0.000000,0.000000,0.000000,269278.0,QTCON-USD
1130863,2024-06-29,0.000000,0.000000,0.000000,0.000000,0.000000,147285.0,QTCON-USD


In [16]:
first_date = megadf['Date'][0]
last_date = megadf['Date'].iloc[-1]

In [17]:
first_date

'2020-10-28'

In [18]:
megadf.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume       float64
Symbol        object
dtype: object

In [19]:
megadf['Date'] = pd.to_datetime(megadf['Date'], format="%Y-%m-%d")

In [20]:
megadf.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume              float64
Symbol               object
dtype: object

In [21]:
os.mkdir('dataset')

In [22]:
megadf.to_csv(f'dataset/1000_cryptos.csv')

# final dataset

In [23]:
df = pd.read_csv(f'/kaggle/working/dataset/1000_cryptos.csv')

In [24]:
df

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,0,2020-10-28,9.946349,10.096945,3.507278,4.387400,4.387400,1444282.0,BTC-USD
1,1,2020-10-29,4.387630,9.351766,2.831383,7.534497,7.534497,2058600.0,BTC-USD
2,2,2020-10-30,7.534420,9.034474,5.401276,7.800243,7.800243,1280547.0,BTC-USD
3,3,2020-10-31,7.800292,8.312695,5.751888,6.257072,6.257072,350813.0,BTC-USD
4,4,2020-11-01,6.257779,7.355800,5.695908,7.217144,7.217144,177058.0,BTC-USD
...,...,...,...,...,...,...,...,...,...
1130860,1130860,2024-06-26,0.000000,0.000000,0.000000,0.000000,0.000000,426785.0,QTCON-USD
1130861,1130861,2024-06-27,0.000000,0.000000,0.000000,0.000000,0.000000,320872.0,QTCON-USD
1130862,1130862,2024-06-28,0.000000,0.000000,0.000000,0.000000,0.000000,269278.0,QTCON-USD
1130863,1130863,2024-06-29,0.000000,0.000000,0.000000,0.000000,0.000000,147285.0,QTCON-USD


# Upload Dataset

# User secrets

In [25]:
user_secrets = UserSecretsClient()
kaggle_apikey = user_secrets.get_secret("kaggle_apikey")
kaggle_username = user_secrets.get_secret("kaggle_username")

os.environ['KAGGLE_USERNAME'] = kaggle_username
os.environ['KAGGLE_KEY'] = kaggle_apikey

# Metadata file

In [26]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print('Starting upload >>>>>>') 
print('Creating metadata file >>>>')
data = {
    "id": "ayushkhaire/top-1000-cryptos-historical"
}
metadata_file_location = '/kaggle/working/dataset/dataset-metadata.json' 
with open(metadata_file_location, 'w', encoding='utf-8') as metadata_file:
    json.dump(data, metadata_file)
print('Metadata file created')

Starting upload >>>>>>
Creating metadata file >>>>
Metadata file created


# upload

In [27]:
retries = 0
while retries < 5:
    try:
        command = "kaggle datasets version -p '/kaggle/working/dataset' -m 'Update' -r zip"
        subprocess.run(command, shell=True, check=True)
        logger.info("Upload complete")
        break
    except Exception as error:
        logger.error(f"Error from Kaggle: {error}")
        time.sleep(5)
        retries += 1

Starting upload for file 1000_cryptos.csv


100%|██████████| 88.7M/88.7M [00:01<00:00, 46.9MB/s]


Upload successful: 1000_cryptos.csv (89MB)
Dataset version is being created. Please check progress at https://www.kaggle.com/ayushkhaire/top-1000-cryptos-historical


# Thank you , and I will see you in next notebook :)